In [ ]:
""" CITATION
@article{scikit-learn,
 title={Scikit-learn: Machine Learning in {P}ython},
 author={Pedregosa, F. and Varoquaux, G. and Gramfort, A. and Michel, V.
         and Thirion, B. and Grisel, O. and Blondel, M. and Prettenhofer, P.
         and Weiss, R. and Dubourg, V. and Vanderplas, J. and Passos, A. and
         Cournapeau, D. and Brucher, M. and Perrot, M. and Duchesnay, E.},
 journal={Journal of Machine Learning Research},
 volume={12},
 pages={2825--2830},
 year={2011}
}

https://towardsdatascience.com/intro-to-support-vector-machines-with-a-trading-example-1d4a7997ced6

"""

In [2]:
from bs4 import BeautifulSoup
import datetime
import json
import numpy as np
import pandas as pd
import requests
import time
import warnings
warnings.simplefilter('ignore')

## Install Homebrew and then install TA-lib, for RSI calculation
import talib as ta
from talib import MA_Type

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm
from sklearn.metrics import make_scorer

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker

In [38]:
raw_data = pd.DataFrame(pd.read_csv('/Users/chico_mua/Desktop/MFE ML/Project/sig_output_4.csv'))
raw_data = raw_data.drop(['Unnamed: 0','ind','X','permno','siccd','shrcls','dlret','close','ret','vol',
                         'WMA','MFI','RSI','Liq','retx','low','high','SMA'],1)
Ticker_List = raw_data['ticker'].unique().tolist()
ind_list = raw_data['industry'].unique().tolist()

# unk_index = raw_data[raw_data['industry'] == 'UNK'].index
# raw_data.loc[unk_index[0]:unk_index[-1],'industry'] = 'Aircraft'

# ind_list = raw_data['industry'].unique().tolist()

In [ ]:
signa_data = raw_data[['']]

In [ ]:
 # Condition of Trading Signal 
for i in Ticker_List:
    Stock_Data = raw_data[raw_data['ticker'] == i]
    Signal_List = []
    for j in Stock_Data[i]['Returns']:
        
        if (j>=0):
            Signal_List.append("1")
            
        else:
            Signal_List.append("0")
            
    Stock_Data[i]['Signal'] = Signal_List


In [ ]:
## data be normalized
## scales each feature by its maximum absolute valu3
max_abs_scaler = preprocessing.MaxAbsScaler()

Model_Dict = {}

## 
date_time_str = '20190101'
split_date = datetime.datetime.strptime(date_time_str, '%Y%m%d')

for i in Ticker_List:
    Stock_Data = raw_data[raw_data['ticker'] == i]
    
    X = np.array(Stock_Data[i].drop(['Signal','Returns'],1))
    X = max_abs_scaler.fit_transform(X)
    Y = np.array(Stock_Data[i]['Signal'])
   
    """
    split test & train data
    
    ## X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    X_sample = X[X['date'] < split_date]
    y_sample = Y[Y['date'] < split_date]
    
    X_out_of_sample = X[X['date'] > split_date]
    y_out_of_sample = Y[Y['date'] > split_date]
    
    """
    
    """"
    Cross Validation
    
    Citation:  https://scikit-learn.org/stable/modules/cross_validation.html
                    
    scoring = {'prec_macro': 'precision_macro',
               'rec_macro': make_scorer(recall_score, average='macro')}
    
    clf 
    scores = cross_validate(clf, X_sample, y_sample, scoring=scoring, cv=5, return_train_score=True)
    
    
    
    
    sorted(scores.keys())
    """
    
    
    scores['train_rec_macro']
    
    Model_Dict[i] = {} #Model_Dict: dict of dict
    Model_Dict[i]['X Train'] = X_sample #key: X Train, value = X_train
    Model_Dict[i]['X Test'] = X_test
    Model_Dict[i]['Y Train'] = y_sample
    Model_Dict[i]['Y Test'] = y_test
    
    model = svm.SVC(kernel='rbf',decision_function_shape='ovo')
    #model = svm.SVC(kernel='linear')
    #model = svm.SVC(kernel='linear',decision_function_shape='ovo')
    #model = svm.SVC(kernel='rbf',decision_function_shape='ovo')
    #model = svm.SVC(kernel='poly')
    #model = svm.SVC(kernel='poly',decision_function_shape='ovo')
    #model = svm.SVC(kernel='sigmoid')
    #model = svm.SVC(kernel='sigmoid',decision_function_shape='ovo')
    
    model.fit(Model_Dict[i]['X Train'], Model_Dict[i]['Y Train'])
    y_pred = model.predict(Model_Dict[i]['X Test'])
    
    Model_Dict[i]['Y Prediction'] = y_pred
    
    #print("SVM Model Info for Ticker: "+i)
    
    #print("Accuracy:",metrics.accuracy_score(Model_Dict[i]['Y Test'], Model_Dict[i]['Y Prediction']))
    Model_Dict[i]['Accuracy'] = metrics.accuracy_score(Model_Dict[i]['Y Test'], Model_Dict[i]['Y Prediction'])
    
    #print("Precision:",metrics.precision_score(Model_Dict[i]['Y Test'], Model_Dict[i]['Y Prediction'],pos_label=str(1),average="macro"))
    Model_Dict[i]['Precision'] = metrics.precision_score(Model_Dict[i]['Y Test'], Model_Dict[i]['Y Prediction'],pos_label=str(1),average="macro")
    
    #print("Recall:",metrics.recall_score(Model_Dict[i]['Y Test'], Model_Dict[i]['Y Prediction'],pos_label=str(1),average="macro"))
    Model_Dict[i]['Recall'] = metrics.recall_score(Model_Dict[i]['Y Test'], Model_Dict[i]['Y Prediction'],pos_label=str(1),average="macro")
    
    #print("#################### \n")

In [ ]:
for i in Ticker_List:
    
    prediction_length = len(Model_Dict[i]['Y Prediction'])
    
    Stock_Data[i]['SVM Signal'] = 0
    Stock_Data[i]['SVM Returns'] = 0
    Stock_Data[i]['Total Strat Returns'] = 0
    Stock_Data[i]['Market Returns'] = 0
    
    Signal_Column = Stock_Data[i].columns.get_loc('SVM Signal') # return column index
    Strat_Column = Stock_Data[i].columns.get_loc('SVM Returns')
    Return_Column = Stock_Data[i].columns.get_loc('Total Strat Returns')
    Market_Column = Stock_Data[i].columns.get_loc('Market Returns')
       
    Stock_Data[i].iloc[-prediction_length:, Signal_Column] = list(map(int, Model_Dict[i]['Y Prediction']))
    Stock_Data[i]['SVM Returns'] = Stock_Data[i]['SVM Signal'] * Stock_Data[i]['Returns'].shift(-1)
    # pair trading recumulative return
    Stock_Data[i].iloc[-prediction_length:,Return_Column] = np.nancumsum(Stock_Data[i]['SVM Returns'][-prediction_length:])
    # benchmark
    Stock_Data[i].iloc[-prediction_length:,Market_Column] = np.nancumsum(Stock_Data[i]['Returns'][-prediction_length:])
    
    Model_Dict[i]['Sharpe_Ratio'] = (Stock_Data[i]['Total Strat Returns'][-1] - Stock_Data[i]['Market Returns'][-1])/ \
                    np.nanstd(Stock_Data[i]['Total Strat Returns'][-prediction_length:])
